In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from secrets import IEX_CLOUD_API_TOKEN

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

In [4]:
price = data['latestPrice']
market_cap = data['marketCap']

In [5]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

In [6]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        "N/A"
    ],
    index = my_columns
    ),
    ignore_index = True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,147.86,2425748263579,N/A


In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )

In [8]:
#final_dataframe

In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = list()
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#   print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,157.320,47644191706,N/A
1,AAL,20.490,13240434541,N/A
2,AAP,210.610,13900190074,N/A
3,AAPL,147.715,2367487067623,N/A
4,ABBV,111.450,201187891983,N/A
...,...,...,...,...
500,YUM,128.860,38294731933,N/A
501,ZBH,144.970,30742218915,N/A
502,ZBRA,507.700,27368056461,N/A
503,ZION,63.011,10120944518,N/A
